In [ ]:
%%capture
import datajoint_plus as djp
djp.config['database.user'] = 'micronsdashboard'
djp.config['database.password'] = 'micronsdashboard#1'

from IPython.core.magics.display import Javascript
from microns_dashboard_api.apps import GetDashboardUser, UserApp, DataJointLoginApp, DataJointTableApp#, ProtocolManager
from microns_dashboard_api.utils import get_user_info_js
from microns_dashboard_api.schemas import dashboard as db
from microns_coregistration_api.dashboard import cell_typer as ctpr
import wridgets as wr
import wridgets.app as wra

In [ ]:
Javascript(get_user_info_js) # must be in its own cell

In [ ]:
user = GetDashboardUser() # must be in its own cell
user

In [ ]:
source = ctpr.database.Protocol.Info * ctpr.database.Submission * ctpr.database.Submission.CellType & 'protocol_name not like "evaluation%"'
submission_table = DataJointTableApp(
    source=source,
    attrs=['submission_id', 'user', 'protocol_name', 'segment_id', 'user_choice', 'cell_type_label', 'note', 'timestamp'],
    n_rows=50
)
def update_submission_table(restrict=None, subtract=None):
    with table_output:
        df = submission_table.to_df(restrict=restrict, subtract=subtract)
        df = df.sort_values('timestamp', ascending=False).rename(columns={'protocol_name': 'protocol'}).reset_index(drop=True)
        display(df, clear=True)

def on_submit(submission_id):
    user = (ctpr.database.Submission.CellType() & {'submission_id': submission_id}).fetch1('user')
    update_submission_table(restrict={'user': user})

table_output = wr.Output()
table = wr.Box(children=[table_output], layout={'width': '100%'})

In [ ]:
header_output = wr.Output()
display(header_output)
def on_user_update():
    with header_output:
        wra.Label(text=f'Welcome, {user.name}.', fontsize=2.5).display()
        
user_app = UserApp(user_app=globals().get('user'), on_user_update=on_user_update)

In [ ]:
app_output = wr.Output()
typer = ctpr.CellTyper(user_app=user_app, on_submit=on_submit)

def on_login(**kwargs):
    with app_output:
        typer.display()
        update_submission_table(restrict={'user': user_app.user})
        
d = DataJointLoginApp(on_login=on_login, hide_on_login=True)
d.display()
display(app_output)
table.display()